# Web-scraping: информация по аренде квартир в Чикаго (ApartmentGuide)

## Импорт библиотек

In [1]:
!pip install requests beautifulsoup4 pandas lxml undetected-chromedriver

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import csv
import json
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random

## Предварительный этап. Тестирование запроса и извлечение данных со страницы  

Посмотрим страницу с отфильтрованными апартаментами в Чикаго

In [3]:
# Задаём URL страницы для парсинга
url = "https://www.apartmentguide.com/apartments/Illinois/Chicago/"

# Указываем заголовки запроса, чтобы сайт думал, что это запрос от браузера
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.7258.128 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

# Отправляем GET-запрос к сайту
r = requests.get(url, headers=headers)

# Проверяем статус ответа (200 = успешно)
print(r.status_code)

# Выводим первые 500 символов страницы
print(r.text[:500])  

200
<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8" data-next-head=""/><meta name="viewport" content="width=device-width" data-next-head=""/><title data-next-head="">Apartments for Rent in Chicago, IL - 12018 Rentals | ApartmentGuide.com</title><link rel="icon" href="/favicon.ico" data-next-head=""/><meta name="format-detection" content="telephone=no" data-next-head=""/><meta name="description" content="Choose from  12018 apartments for rent in Chicago, Illinois by comparing verified rati


In [4]:
# Создаём объект BeautifulSoup для удобного парсинга HTML
soup = BeautifulSoup(r.text, 'lxml')

In [5]:
# Находим блок с адресом карточки
adress = soup.find('li', class_='_90363ab6').find('a')['href']
adress

'/a/1433-45-W-Lunt-Ave-Chicago-IL-6021040/'

In [6]:
# Ссылка на карточку
link = 'https://www.apartmentguide.com' + soup.find('li', class_='_90363ab6').find('a')['href']
link

'https://www.apartmentguide.com/a/1433-45-W-Lunt-Ave-Chicago-IL-6021040/'

Посмотрим элементы в карточке с описанием апартаментов

In [7]:
# Задаём URL страницы для парсинга
url_2 = 'https://www.apartmentguide.com/a/7444-N-Seeley-Chicago-IL-6003150/'

# Указываем заголовки запроса, чтобы сайт думал, что это запрос от браузера
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.7258.128 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

# Отправляем GET-запрос к сайту
r_2 = requests.get(url_2, headers=headers)

# Проверяем статус ответа (200 = успешно)
print(r_2.status_code)

# Выводим первые 500 символов страницы
print(r_2.text[:500])

200
<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8" data-next-head=""/><meta name="viewport" content="width=device-width" data-next-head=""/><title data-next-head="">7444 N. Seeley - Chicago, IL 60645 | ApartmentGuide.com</title><link rel="icon" href="/favicon.ico" data-next-head=""/><meta name="format-detection" content="telephone=no" data-next-head=""/><meta name="description" content="Find your new home at 7444 N. Seeley located at 7444 N Seeley Ave, Chicago, IL 60645. Floor plans star


In [8]:
# Создаём объект BeautifulSoup для удобного парсинга HTML
soup_2 = BeautifulSoup(r_2.text, 'lxml')

In [9]:
# Находим блок с ценой
price = soup_2.find('div', class_= '_51bb0611').find('div', {'data-tid': 'pdpKeyInfo_price'}).text.strip()
price

'$1,455+'

In [10]:
# Находим блок с количеством комнат
bads = soup_2.find('li', {'data-tid': 'pdpKeyInfo_bedText'}).text.strip()
bads

'1–3 Beds'

In [11]:
# Находим блок с количеством ванных комнат
baths = soup_2.find('li', {'data-tid': 'pdpKeyInfo_bathText'}).text.strip()
baths

'1 Bath'

In [12]:
# Находим блок с размером апартаментов в sqft
try:
    sqft = soup_2.find('li', {'data-tid': 'pdpKeyInfo_sqFt'}).text.strip()
except AttributeError:
    sqft = 0

In [13]:
sqft

0

In [14]:
# Находим блок "Pet policy"
pets = [s.get_text(strip=True)
        for s in soup_2.find('div', attrs={'data-tag_section': 'pet_policy_summary'}) \
                     .find_all('span')]

In [15]:
pets

['Cats OK']

In [16]:
# Находим блоки с описанием всех удобств в апартаментах 
sections = ['Kitchen', 'Features', 'Unique', 'Parking']
amenities_data = {}

for section in sections:
    try:
        block = soup_2.find('h3', text=section).find_next('ul')
        items = [li.text.strip() for li in block.find_all('li')]
        amenities_data[section] = ', '.join(items) if items else 0
    except AttributeError:
        amenities_data[section] = 0

for k, v in amenities_data.items():
    print(k, ':', v)

Kitchen : Dishwasher
Features : 0
Unique : Dryer, Heating: Gas, On-Site Management, Pet Friendly, Washer
Parking : 0


/var/folders/pj/zsgv91ds4gzcy93zwc1mwl280000gn/T/ipykernel_54203/178365932.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  block = soup_2.find('h3', text=section).find_next('ul')


In [17]:
# Находим блок со сроком аренды
try:
    term_length = soup_2.find('h3', string='Term Length').find_next('div').text.strip()
except:
    term_length = 0

print('Term Length =', term_length)

Term Length = 12-Month


In [18]:
# Находим JSON-LD блок со структурированными данными на странице
script = soup_2.find("script", {"id": "json-ld", "type": "application/ld+json"})

# Загружаем содержимое скрипта как JSON
data = json.loads(script.string)

# Извлекаем координаты апартаментов: широту и долготу
latitude = data["mainEntity"]["geo"]["latitude"]
longitude = data["mainEntity"]["geo"]["longitude"]

print(f"Latitude: {latitude}, Longitude: {longitude}")

Latitude: 42.017309, Longitude: -87.6818


In [19]:
# Находим тег <script> с JSON-LD, в котором лежат структурированные данные об объекте
script_tag = soup_2.find('script', {'id': 'json-ld', 'type': 'application/ld+json'})

# Преобразуем содержимое тега в Python-словарь
data = json.loads(script_tag.string)

# Извлекаем почтовый индекс апартаментов из раздела address
postal_code = data['mainEntity']['address']['postalCode']
print(postal_code)

60645


## Скрейпер для всех страниц и карточек ApartmentGuide

In [ ]:
# Настройки undetected-chromedriver
# Создаём опции для браузера: размер окна и отключение признаков автоматизации
options = uc.ChromeOptions()
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")

# Запускаем браузер
driver = uc.Chrome(options=options)

# Базовый URL и переменные для хранения данных
base_url = "https://www.apartmentguide.com/apartments/Illinois/Chicago/page-{}/"
data = []
page = 1

# Основной цикл по страницам
while True:
    # Формируем URL для текущей страницы
    url = base_url.format(page) if page > 1 else "https://www.apartmentguide.com/apartments/Illinois/Chicago/"
    print(f"Парсим страницу {page}: {url}")
    driver.get(url)

    # Скроллим страницу для подгрузки карточек
    last_height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, last_height, random.randint(200, 400)):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.2, 0.5))

    # Ждём, пока карточки появятся на странице
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li[data-tid^='srp_card_']"))
        )
    except:
        print("Карточки не найдены, выходим.")
        break

    # Создаём объект BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(driver.page_source, "lxml")
    cards = soup.select("li[data-tid^='srp_card_']")
    if not cards:
        print("Карточки закончились, выходим.")
        break

    print(f"Найдено карточек: {len(cards)}")

    # Сбор ссылок на каждую карточку квартиры
    card_links = []
    for card in cards:
        a = card.find("a")
        if a and a.get("href"):
            link = a["href"]
            if not link.startswith("http"):
                link = "https://www.apartmentguide.com" + link
            card_links.append(link)

    # Парсим каждую карточку отдельно
    for link in card_links:
        driver.get(link)
        time.sleep(random.uniform(1, 2))
        soup2 = BeautifulSoup(driver.page_source, "lxml")

        # Основные данные: цена, спальни, ванные, площадь
        try: price = soup2.select_one("div[data-tid='pdpKeyInfo_price']").get_text(strip=True)
        except: price = None
        try: beds = soup2.select_one("li[data-tid='pdpKeyInfo_bedText']").get_text(strip=True)
        except: beds = None
        try: baths = soup2.select_one("li[data-tid='pdpKeyInfo_bathText']").get_text(strip=True)
        except: baths = None
        try: sqft = soup2.select_one("li[data-tid='pdpKeyInfo_sqFt']").get_text(strip=True)
        except: sqft = None

        # Политика по животным
        try:
            pets = ", ".join([e.get_text(strip=True) for e in soup2.select("div[data-tag_section='pet_policy_summary'] span")])
        except: pets = None

        # Удобства квартиры
        amenities_data = {}
        for section in ['Kitchen', 'Features', 'Unique', 'Parking']:
            try:
                ul = soup2.find("h3", string=section).find_next("ul")
                items = [li.get_text(strip=True) for li in ul.find_all("li")]
                amenities_data[section] = ", ".join(items) if items else None
            except:
                amenities_data[section] = None

        # Срок аренды
        try:
            term_length = soup2.find("h3", string="Term Length").find_next("div").get_text(strip=True)
        except:
            term_length = None

        # Координаты и почтовый индекс из JSON-LD
        try:
            script = soup2.find("script", id="json-ld", type="application/ld+json").string
            latitude = script.split('"latitude":')[1].split(",")[0].strip().strip('"')
            longitude = script.split('"longitude":')[1].split(",")[0].strip().strip('"')
            postal_code = script.split('"postalCode":')[1].split(",")[0].strip().strip('"')
        except:
            latitude = longitude = postal_code = None

        # Добавляем данные в общий список
        data.append([
            price, beds, baths, sqft, pets,
            amenities_data['Kitchen'], amenities_data['Features'],
            amenities_data['Unique'], amenities_data['Parking'],
            term_length, latitude, longitude, postal_code, link
        ])
        print(f"✓ {link}")
        print(f"   Price: {price} | Beds: {beds} | Baths: {baths} | Sqft: {sqft} | Pets: {pets}")
        print(f"   Term: {term_length} | Lat: {latitude} | Lon: {longitude} | ZIP: {postal_code}")

    # Делаем паузу перед следующей страницей
    time.sleep(random.uniform(3, 5))
    page += 1

# Закрываем браузер после завершения сбора данных
driver.quit()

In [ ]:
# Преобразуем собранные данные в DataFrame pandas с понятными названиями колонок
data = pd.DataFrame(data, columns=[
    'price','beds','baths','sqft','pets',
    'Kitchen','Features','Unique','Parking',
    'term_length','latitude','longitude','postal_code','link'
])

# Сохраняем DataFrame в CSV-файл для дальнейшей обработки и анализа
data.to_csv('chicago_rent.csv', index=False, encoding="utf-8-sig")

print("Сбор завершён. Всего квартир:", len(data))